# Capstone Project - Segmenting & Clustering Neighbourhoods in Toronto - Part 2

Author : Avinandan Mukherjee

In [8]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


### Performing steps again from Part 1 to build the dataframe again

In [10]:
url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_Toronto =pd.read_html(url)
df_Toronto[0].to_csv('toronto.csv')
df_Toronto =pd.read_csv('toronto.csv')
df_Toronto.head()

,Unnamed: 0,Postal Code,Borough,Neighbourhood
0,0,M1A,Not assigned,Not assigned
1,1,M2A,Not assigned,Not assigned
2,2,M3A,North York,Parkwoods
3,3,M4A,North York,Victoria Village
4,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [11]:
df_Toronto.drop('Unnamed: 0',inplace =True, axis =1)
df_Toronto.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [15]:
df_Toronto['Postal Code'].replace({'Not assigned':np.NaN},inplace =True)
df_Toronto.dropna(inplace =True)
df_Toronto =df_Toronto.reset_index(drop=True)
df_Toronto

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [13]:
df_Toronto.shape

(180, 3)

### Part 2 : Adding GeoSpatial Coordinates to Postal Codes

Using the given link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [51]:
#toronto_loc = 'https://cocl.us/Geospatial_data'
!wget -q -O 'Geospatial_data.csv'  https://cocl.us/Geospatial_data
df_GeoLocation = pd.read_csv('Geospatial_data.csv')
df_GeoLocation.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Combining the Toronto dataset and Geolocation dataset with mapped Postal Codes

In [54]:
df_Toronto =df_Toronto.sort_values('Postal Code').reset_index()

In [55]:
df_Toronto.drop('index',inplace =True,axis =1)

In [57]:
df_Toronto =pd.merge(df_Toronto, df_GeoLocation, on ='Postal Code')

In [58]:
df_Toronto.head()

,level_0,Postal Code,Borough,Neighbourhood,Latitude_x,Longitude_x,Latitude_y,Longitude_y
0,0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,43.806686,-79.194353
1,1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,43.773136,-79.239476
